# EDA

In [99]:
import psycopg2 as ps
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
import numpy as np

In [2]:
def conexion_BBDD(nombre_BBDD, usuario, contraseña, anfitrion, puerto):

    conn = ps.connect(
                    dbname = nombre_BBDD, 
                    user = usuario,
                    password = contraseña,
                    host = anfitrion,
                    port = puerto)

    return conn

In [3]:
dbname = "BBDD_Hipica", # base a la que nos queremos conectar
user = "postgres",
password = "admin",
host = "localhost",
port = "5432" # puerto en el que s eencuentra postgres

In [4]:
conn = conexion_BBDD(dbname[0], user[0], password[0], host[0], port)
cur = conn.cursor()
# COmprobamos que la conexión está creada y conectada
cur.execute("SELECT version();")
cur.fetchone() 

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)

## Análisis de concursos y pruebas

In [5]:
def ejecutor_querys(cur, query):
    cur.execute(query)
    return cur.fetchall()

In [6]:
conn.rollback()

In [63]:
def concursos_seleccionado(jinete):
    query = f""" 
                SELECT 
                    DISTINCT c.nombre_caballo
                FROM resultados r
                    JOIN caballos c ON r.id_caballo = c.id_caballo
                    JOIN jinetes j ON r.id_jinete = j.id_jinete
                WHERE j.nombre_jinete = '{jinete}';
        """
    return ejecutor_querys(cur, query)

caballos = list(concursos_seleccionado('hugo álvarez amaro'))
caballos

[('caloua quick ps',),
 ('casaspezia',),
 ('el especal particular kc z',),
 ('idiego van het kapoelhof z',),
 ('juno de llamosas',),
 ('milme',),
 ('o-tanagra',),
 ('oswald b',),
 ('pillot bmg',),
 ('thunderbird van de derkes lp',),
 ("umbreo van d'abdijhoeve",),
 ('usus dc',)]

In [87]:
import re
def extraer_altura_y_edad(texto):
    altura = None
    edad = None

    match_altura = re.search(r'(\d{1,2}[,.]\d{2})\s?(?:m\.?)?', texto)
    if match_altura:
        altura = match_altura.group(1).replace(',', '.')

    match_edad = re.search(r'\b([5-8])\s?años\b', texto, flags=re.IGNORECASE)
    if match_edad:
        edad = f"{match_edad.group(1)} años"

    return altura, edad


In [56]:
conn.rollback()

In [129]:
query_prueba = """ 
        SELECT 
            j.nombre_jinete,
            c.nombre_caballo,
            rs.puntos_obs_r1,
            rs.puntos_tmp_r1,
            rs.tiempo_r1,
            rs.puntos_obs_r2,
            rs.puntos_tmp_r2,
            rs.tiempo_r2,
            rs.puntos_obs_r3,
            rs.puntos_tmp_r3,
            rs.tiempo_r3,
            p.nombre_prueba,
            co.nombre_concurso,
            r.estado,
            p.fecha_prueba,
            r.puesto
        FROM resultados r
            JOIN caballos c ON r.id_caballo = c.id_caballo
            JOIN jinetes j ON r.id_jinete = j.id_jinete
            JOIN resultados_salto rs ON r.id_resultado = rs.id_resultado
            JOIN pruebas p ON r.id_prueba = p.id_prueba
            JOIN concursos co ON r.id_concurso = co.id_concurso
        WHERE j.nombre_jinete = 'hugo álvarez amaro' AND c.nombre_caballo = 'casaspezia';
"""
binomio = pd.DataFrame(ejecutor_querys(cur, query_prueba)).rename(columns = {0: 'jinete', 1: 'caballo', 2: 'puntos_obs_r1', 3: 'puntos_tmp_r1', 4: 'tiempo_r1',
                                                                   5: 'puntos_obs_r2', 6: 'puntos_tmp_r2', 7: 'tiempo_r2',
                                                                   8: 'puntos_obs_r3', 9: 'puntos_tmp_r3', 10: 'tiempo_r3',
                                                                   11: 'prueba', 12: 'concurso', 13: 'estado', 14: 'fecha_prueba', 15: 'puesto'})
# METRICAS
n_concursos = len(binomio['concurso'].unique()) # numero de concursos en los que el caballo ha competido con el jinete seleccionado
n_caballos_corridos = len(caballos) # numero de cabllos que el jinete seleccionado corre actualmente/ha corrido este año
porcentaje_recorridos_finalizados = round(len(binomio[binomio["estado"] == "FIN"])/len(binomio) * 100, 2) # porcentaje de pruebas finalizadas

tipos_pruebas_altura = binomio['prueba'].apply(extraer_altura_y_edad)
alturas = []
edad = []
for elemento in tipos_pruebas_altura:
    if pd.notna(elemento[0]):
        alturas.append(elemento[0])
    if pd.notna(elemento[1]):
        edad.append(elemento[1])
    else:
        continue
alturas_buenas = list(set(alturas)) # alturas en las que el jinete ha competido con el caballo seleccionado
edad_bueno = list(set(edad)) # si es caballo joven o no 


tiempos = binomio[['tiempo_r1', 'tiempo_r2', 'tiempo_r3']].apply(pd.to_numeric, errors='coerce')
promedio_tiempo = round(tiempos.stack().mean(), 2) # promedio de tiempo que realiza el caballo en un recorrido

# Solo filas con estado FIN (salida a pista válida)
df_fin = binomio[binomio['estado'] == 'FIN']

# Vamos a revisar todas las rondas para contar las salidas y calcular puntos
rondas = ['r1', 'r2', 'r3']

# Creamos listas para guardar resultados de cada salida a pista (cada ronda válida)
salidas_pista = []

for _, fila in df_fin.iterrows():
    for r in rondas:
        p_obs = fila[f'puntos_obs_{r}']
        p_tmp = fila[f'puntos_tmp_{r}']
        tiempo = fila[f'tiempo_{r}']
        
        # Comprobar si salió a pista: alguna de estas 3 columnas tiene un valor numérico válido
        if pd.notna(p_obs) or pd.notna(p_tmp) or pd.notna(tiempo):
            # Asegurarnos que p_obs sea número, sino 0 para contar correctamente
            p_obs_val = p_obs if pd.notna(p_obs) else 0
            salidas_pista.append(p_obs_val)

# Número de salidas a pista
num_salidas = len(salidas_pista) # numero de recorridos/salidas a pista que ha realizado el cabllo con el jinete seleccionado

# Promedio de puntos en obstáculos por salida
promedio_puntos_obs = round(np.mean(salidas_pista)) # promedio de puntos de obstaculos que realiza el caballo en una salida a pista 

# Porcentaje de salidas con 0 puntos en obstáculos
veces_cero = sum(1 for x in salidas_pista if x == 0) # numero de veces que el caballo ha hecho cero puntos en obstaculos
promedio_veces_cero = (veces_cero / num_salidas if num_salidas > 0 else np.nan)*100
jinete = binomio['jinete'].unique()[0]
caballo = binomio['caballo'].unique()[0]
edad_caballo = edad_bueno[0] if edad_bueno else "No joven"
promedio_puntos_obs, promedio_veces_cero, alturas_buenas, edad_bueno, n_concursos, num_salidas, veces_cero


(4, 50.0, ['1.45', '1.50', '1.55', '1.35'], [], 6, 14, 7)

In [128]:
tabla_md = f"""
| Métrica               | Valor                        |
|-----------------------|------------------------------|
| Jinete                | {jinete}                     |
| Caballo               | {caballo}                    |
| Edad caballo          | {edad_caballo}               |
| Concursos realizados  | {n_concursos}                |
| Altura óbstaculos     | {alturas_buenas}             |
| Pots_medios_salida    | {promedio_puntos_obs}        |
| Promedio_cero_obs     | {promedio_veces_cero}        |
"""

print(tabla_md)



| Métrica               | Valor                        |
|-----------------------|------------------------------|
| Jinete                | hugo álvarez amaro                     |
| Caballo               | casaspezia                    |
| Edad caballo          | No joven               |
| Concursos realizados  | 6                |
| Altura óbstaculos     | ['1.45', '1.50', '1.55', '1.35']             |
| Pots_medios_salida    | 4        |
| Promedio_cero_obs     | 50.0        |



In [130]:
colores = ['#4c78a8', '#54a24b']
estado_counts = binomio['estado'].value_counts().reset_index()
estado_counts.columns = ['estado', 'count']
fig = px.pie(estado_counts, values="count", names="estado", title='Porcentaje de finalización pruebas', color_discrete_sequence=colores)
fig.update_traces(textinfo='percent', textfont_color='white')
fig.update_layout(width=600, height=400, title_x=0.5, title_font=dict(size = 16, weight='bold'))
fig.show()

In [133]:


# Nos aseguramos de que la fecha sea tipo datetime
binomio['fecha_prueba'] = pd.to_datetime(binomio['fecha_prueba'])

# Creamos una nueva columna con la suma total de puntos por obstáculos
binomio['puntos_obs_totales'] = binomio[['puntos_obs_r1', 'puntos_obs_r2', 'puntos_obs_r3']].fillna(0).sum(axis=1)

# Agrupamos por fecha de prueba y sumamos los puntos de obstáculos por día
df_grouped = binomio.groupby('fecha_prueba', as_index=False)['puntos_obs_totales'].sum()

# Ordenamos por fecha
df_grouped = df_grouped.sort_values('fecha_prueba')

# Creamos el gráfico de barras
fig = px.bar(
    df_grouped,
    x='fecha_prueba',
    y='puntos_obs_totales',
    title='Puntos en obstáculos por fecha de prueba',
    labels={
        'fecha_prueba': 'Fecha de prueba',
        'puntos_obs_totales': 'Puntos por obstáculos'
    },
    color='puntos_obs_totales',
    color_continuous_scale='Reds'
)

fig.update_layout(xaxis_title='Fecha', yaxis_title='Puntos de obstáculos')
fig.show()

C:\Users\gabri\AppData\Local\Temp\ipykernel_24840\4008578131.py:5: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [118]:
binomio

,jinete,caballo,puntos_obs_r1,puntos_tmp_r1,tiempo_r1,puntos_obs_r2,puntos_tmp_r2,tiempo_r2,puntos_obs_r3,puntos_tmp_r3,tiempo_r3,prueba,concurso,estado,fecha_prueba,puesto
0,hugo álvarez amaro,casaspezia,0.0,NaN,67.78,NaN,None,NaN,None,None,None,"1,50 (G) - A c.c. (238.2.1)",CSI4* - 1st Week XXXI Andalucía Sunshine Tour ...,FIN,2025-02-06,14.0
1,hugo álvarez amaro,casaspezia,12.0,0.0,70.51,NaN,None,NaN,None,None,None,"1,50 (G) - A c.c. (238.2.1)",CSI4* - 1st Week XXXI Andalucía Sunshine Tour ...,FIN,2025-02-07,42.0
2,hugo álvarez amaro,casaspezia,4.0,0.0,75.78,NaN,None,NaN,None,None,None,"1,55 (GP) - A.c.c. y desempate (238.2.2)",CSI4* - 1st Week XXXI Andalucía Sunshine Tour ...,FIN,2025-02-09,12.0
3,hugo álvarez amaro,casaspezia,8.0,0.0,70.99,NaN,None,NaN,None,None,None,"1,50 (GP) - A.c.c. y desempate (238.2.2)",CSI4* - 2nd Week XXXI Andalucía Sunshine Tour ...,FIN,2025-02-14,52.0
4,hugo álvarez amaro,casaspezia,4.0,0.0,79.24,NaN,None,NaN,None,None,None,"1,50 (GP) - A.c.c. y desempate (238.2.2)",CSI4* - 2nd Week XXXI Andalucía Sunshine Tour ...,FIN,2025-02-16,27.0
5,hugo álvarez amaro,casaspezia,NaN,NaN,NaN,NaN,None,NaN,None,None,None,"1,50 (G) - A c.c. (238.2.1)",CSI4* - 3rd Week XXXI Andalucía Sunshine Tour ...,RET,2025-02-21,NaN
6,hugo álvarez amaro,casaspezia,NaN,NaN,NaN,NaN,None,NaN,None,None,None,"1,45 (GP) - A.c.c. y desempate (238.2.2)",CSI4* - 3rd Week XXXI Andalucía Sunshine Tour ...,RET,2025-02-22,NaN
7,hugo álvarez amaro,casaspezia,0.0,NaN,61.33,NaN,None,NaN,None,None,None,"1,35 (G) - A c.c. (238.2.1)",CSI4* - 6th Week XXXI Andalucía Sunshine Tour ...,FIN,2025-03-13,1.0
8,hugo álvarez amaro,casaspezia,0.0,NaN,74.20,NaN,None,NaN,None,None,None,"1,50 (GP) - Dos Mangas (273.3.4)",CSI4* - 6th Week XXXI Andalucía Sunshine Tour ...,FIN,2025-03-14,10.0
9,hugo álvarez amaro,casaspezia,4.0,0.0,77.29,NaN,None,NaN,None,None,None,"1,55 (GP) - A.c.c. y desempate (238.2.2)",CSI4* - 6th Week XXXI Andalucía Sunshine Tour ...,FIN,2025-03-16,23.0


In [68]:
query_pruebas = """
        
        SELECT DISTINCT
            nombre_prueba
        FROM pruebas p; 

"""
ejecutor_querys(cur, query_pruebas)

[('1,30 - A.s.c. y desempate al cronómetro (238.1.2)',),
 ('1,20 - Velocidad y Manejabilidad (263)',),
 ('Ch - 1,30 m. - Dos Fases (274.1.5.3)',),
 ('Ch - 1,25 m. - 2 fases Asc / Acc (274.1.5.2)',),
 ('J - 1,35 m. - 2 fases Asc / Acc (274.1.5.2)',),
 ('CLASSIC TOUR A - 2 fases Asc / Acc (274.1.5.2)',),
 ('6 años - 1,25 - A.c.c. y desempate (238.2.2)',),
 ('7 años - 1,30 - Dos Fases Especial (274.2)',),
 ('Ch - 1,30 m. - A.c.c. y desempate (238.2.2)',),
 ('J - 1,35 m. - Dos Fases (274.1.5.3)',),
 ('1,30 (P) - A c.c. (238.2.1)',),
 ('6 años - 1,30 - A c.c. (238.2.1)',),
 ('1,45 - Dos Fases Especial (274.2)',),
 ('1,25 - Dos Fases (274.1.5.3)',),
 ('5 años - 1,10 - A c.c. (238.2.1)',),
 ('J - 1,35 m. - A c.c. (238.2.1)',),
 ('1,30 - A.c.c. y desempate al cronómetro (238.2.2)',),
 ('YR - 1,40 m. - 2 fases Asc / Acc (274.1.5.2)',),
 ('1,10 - A.c.c. y desempate (238.2.2)',),
 ('1,40 (G) - A.c.c. y desempate (238.2.2)',),
 ('1,20 - Dos Fases (274.1.5.3)',),
 ('1,40 (G) - A c.c. (238.2.1)',),


In [8]:
query_concursos_completos = """
        SELECT *
        FROM concursos c;
"""
query_n_concursos = """
        SELECT COUNT(id_concurso)
        FROM concursos c;
"""
query_fechas_concursos = """
        SELECT 
                MIN(fecha_inicio_concurso),
                MAX(fecha_fin_concurso)
        FROM concursos c;
"""
query_concursos_mes = """
        SELECT 
                EXTRACT(MONTH FROM fecha_inicio_concurso),
                COUNT(id_concurso)
        FROM concursos c
        GROUP BY EXTRACT(MONTH FROM fecha_inicio_concurso)
        ORDER BY COUNT(id_concurso) DESC;
"""
query_concursos_ambito = """
        SELECT 
                ambito_concurso,
                CONCAT((COUNT(id_concurso)) * 100 / (SELECT COUNT(id_concurso) FROM concursos c), ' ', '%')
        FROM concursos c
        GROUP BY ambito_concurso
        ORDER BY COUNT(id_concurso) DESC;
"""
query_concursos_federacion = """
        SELECT 
                federacion_concurso,
                COUNT(id_concurso)
        FROM concursos c
        GROUP BY federacion_concurso
        ORDER BY COUNT(id_concurso) DESC;
"""
query_concursos_provincia = """
        SELECT 
                provincia_concurso,
                COUNT(id_concurso)
        FROM concursos c
        GROUP BY provincia_concurso
        ORDER BY COUNT(id_concurso) DESC;
"""

query_concursos_localidad = """
        SELECT 
                localidad_concurso,
                COUNT(id_concurso)
        FROM concursos c
        GROUP BY localidad_concurso
        ORDER BY COUNT(id_concurso) DESC;
"""

query_concursos_localidad = """
        SELECT 
                localidad_concurso,
                COUNT(id_concurso)
        FROM concursos c
        GROUP BY localidad_concurso
        ORDER BY COUNT(id_concurso) DESC;
"""
query_concursos_pruebas_concurso = """
        
        WITH pruebas_concurso AS (
                SELECT 
                        c.id_concurso,
                        COUNT(DISTINCT p.id_prueba) as n_pruebas_concurso
                FROM resultados r
                        JOIN concursos c ON r.id_concurso = c.id_concurso
                        JOIN pruebas p ON r.id_prueba = p.id_prueba
                GROUP BY c.id_concurso
        )
        SELECT 
                ROUND(AVG(n_pruebas_concurso), 0)
        FROM pruebas_concurso;

"""

query_pruebas = """
        
        SELECT 
                DISTINCT SUBSTRING(
                         nombre_prueba 
                         FROM POSITION('(2' IN nombre_prueba)
                         ) as tipo_prueba, 
                COUNT(id_prueba)
        FROM pruebas p
        GROUP BY tipo_prueba
        ORDER BY 2 DESC; 

"""
query_duracion_concursos = """
        
        SELECT 
                ROUND(AVG(fecha_fin_concurso::date - fecha_inicio_concurso::date), 0)
        FROM concursos c; 

"""

query_dinero_repartido_concursos = """
        
        WITH ConteoJinetesCaballosPorConcurso AS (
        SELECT
                r.id_concurso,
                COUNT(DISTINCT r.id_jinete) AS conteo_jinetes,
                COUNT(DISTINCT r.id_caballo) AS conteo_caballos
        FROM resultados r
        GROUP BY r.id_concurso
        ),
        DineroTotalPorConcurso AS (
        SELECT
                r.id_concurso,
                SUM(r.dinero_premio) AS total_premio_concurso
        FROM resultados r
        GROUP BY r.id_concurso
        )
        SELECT
                c.categoria_concurso,
                AVG(cjcc.conteo_jinetes) AS media_jinetes,
                AVG(cjcc.conteo_caballos) AS media_caballos,
                AVG(dtpc.total_premio_concurso) AS media_dinero_premio_por_concurso
        FROM concursos c
                JOIN ConteoJinetesCaballosPorConcurso cjcc ON c.id_concurso = cjcc.id_concurso
                JOIN resultados r ON c.id_concurso = r.id_concurso
                JOIN DineroTotalPorConcurso dtpc ON c.id_concurso = dtpc.id_concurso
        GROUP BY c.categoria_concurso
        ORDER BY 4 DESC;

"""

In [9]:
meses = {
    1: "enero",
    2: "febrero",
    3: "marzo",
    4: "abril",
    5: "mayo",
    6: "junio",
    7: "julio",
    8: "agosto",
    9: "septiembre",
    10: "octubre",
    11: "noviembre",
    12: "diciembre"
}
prueba_norma = {
    '(238.2.1)': 'Baremo A con cronómetro',
    '(238.2.2)': 'Baremo A con cronómetro y desempate',
    '(274.1.5.3)': 'Dos fases, ambas con cronómetro',
    '(274.2)': 'Dos fases especial',
    '(238.1.1)': 'Baremo A sin cronómetro',
    '(274.1.5.2)': 'Dos fases, primera sin cronómetro, segunda con cronómetro',
    '(269.3.2)': 'Prueba de potencia',
    '(263)': 'Baremo C',
    '(273.3.4)': 'Prueba con dos mangas y desempate',
    '(239)': 'Baremo A juzgado como Baremo C',
    '(264)': 'Prueba de caza',
    '(273.3.3.1)': 'Prueba con dos mangas sin desempate',
    '(276.2)': 'Prueba con puntuación progresiva',
    '(238.1.2)': 'Baremo A sin cronómetro con desempate'
}



In [10]:
conn.rollback()

cuantos caballos por categoría, media de caballos por concurso, y por categoría, concursos que mas caballos han atraído, dinero repartido en premios de media en un concurso de cada categoría, 

In [11]:
caballos_jinetes_categoria = pd.DataFrame(ejecutor_querys(cur, query_dinero_repartido_concursos))
caballos_jinetes_categoria[1] = caballos_jinetes_categoria[1].apply(lambda x: round(x, 2))
caballos_jinetes_categoria[2] = caballos_jinetes_categoria[2].apply(lambda x: round(x, 2))
caballos_jinetes_categoria[3] = caballos_jinetes_categoria[3].apply(lambda x: round(x, 2))
caballos_jinetes_categoria = caballos_jinetes_categoria.rename(columns = {0: "categoria", 1: "jinetes", 2: "caballos", 3: "dinero_medio"})
caballos_jinetes_categoria

,categoria,jinetes,caballos,dinero_medio
0,CSI****,335.25,809.76,393141.89
1,CSI***,113.11,270.81,262111.37
2,CSI**,121.86,286.88,91725.00
3,CSI*,143.44,244.70,83613.54
4,CSI YH*,154.20,348.28,53165.94
5,CSN*****,175.92,286.92,37879.63
6,CSN****,129.00,237.00,26215.00
7,CSIO Ch/J/Y,77.00,108.00,22900.00
8,CSN***,77.33,134.87,13119.28
9,CSN**,124.69,179.20,6156.57


In [12]:
tipos_pruebas = ejecutor_querys(cur, query_pruebas)
tipos_pruebas 

[('(238.2.1)', 450),
 ('(238.2.2)', 205),
 ('(274.1.5.3)', 198),
 ('(274.2)', 126),
 ('(238.1.1)', 87),
 ('(274.1.5.2)', 33),
 ('(269.3.2)', 15),
 ('(263)', 9),
 ('(273.3.4)', 9),
 ('(239)', 2),
 ('(264)', 2),
 ('(273.3.3.1)', 2),
 ('(276.2)', 2),
 ('(238.1.2)', 1)]

In [13]:
federaciones = ejecutor_querys(cur, query_concursos_federacion)

In [14]:
concursos_provincias = ejecutor_querys(cur, query_concursos_provincia)
concursos = pd.DataFrame(concursos_provincias)
concursos[2] = round((concursos[1]/concursos[1].sum())*100, 2)
concursos

,0,1,2
0,Valencia,75,61.48
1,Cádiz,19,15.57
2,Madrid,7,5.74
3,Sevilla,4,3.28
4,Barcelona,4,3.28
5,Tenerife,3,2.46
6,Granada,2,1.64
7,Cantabria,2,1.64
8,Zamora,2,1.64
9,Asturias,1,0.82


In [15]:
# Convert concursos_provincias to DataFrame for plotting
df_provincias = pd.DataFrame(concursos_provincias, columns=['Provincia', 'Nº concursos'])

fig = px.bar(df_provincias, x='Provincia', y='Nº concursos', title = "Concursos por provincia")
fig.update_layout(
    width=800, 
    height=400,
    title_font=dict(size = 15, weight='bold'),
    title_x=0.5,
    xaxis_title=dict(text='Provincia', font=dict(size = 12, weight='bold')),
    yaxis_title=dict(text='Nº concursos', font=dict(size = 12, weight='bold')),
    plot_bgcolor='white'
)
fig.show()

In [16]:
localidades = ejecutor_querys(cur, query_concursos_localidad)

In [17]:
df_mes = ejecutor_querys(cur, query_concursos_mes)
df = pd.DataFrame(df_mes)
df = df.sort_values(by=0, ascending=True)
df[0] = df[0].apply(lambda x: meses[int(x)])
df
# cosita = pd.DataFrame(cosos)
# pd.DataFrame(cosita.groupby(8)[0].count())

,0,1
2,enero,27
0,febrero,41
1,marzo,33
3,abril,21


In [18]:
import plotly.graph_objects as go

In [19]:
fig = px.line(df, x=0, y=1, title='Concursos por mes')
fig.update_layout(
    title_font=dict(size = 20, weight='bold'),
    title_x=0.45,
    xaxis_title=dict(text='Fecha', font=dict(weight='bold')),
    yaxis_title=dict(text='Valor', font=dict(weight='bold')),
    plot_bgcolor='white',
    yaxis=dict(showgrid=True, gridcolor='lightgray', showticklabels=False),
    # xaxis=dict(showline=True, tickfont=dict(weight='bold')),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)
x_data = fig.data[0]['x']
y_data = fig.data[0]['y']

# Añade una nueva traza con los números
fig.add_trace(go.Scatter(x=x_data, y=y_data,
                         mode='text',
                         text=y_data,
                         textposition="top center",
                         showlegend=False,
                         textfont=dict(weight='bold')))
fig.show()

In [20]:
n_concursos = ejecutor_querys(cur, query_n_concursos)
fechas_concursos_25 = ejecutor_querys(cur, query_fechas_concursos)
n_concursos_mes = ejecutor_querys(cur, query_concursos_mes)
n_concursos_ambito= ejecutor_querys(cur, query_concursos_ambito)
media_pruebas_concurso = ejecutor_querys(cur, query_concursos_pruebas_concurso)
tipos_pruebas = ejecutor_querys(cur, query_pruebas)

print(f"""
      Se han celebrado un total de {n_concursos[0][0]} concursos, entre el {str(fechas_concursos_25[0][0])} y el {str(fechas_concursos_25[0][1])}. Un {n_concursos_ambito[0][1]} de los concursos han sido de 
      ámbito {str(n_concursos_ambito[0][0]).lower()}, frente a los de ámbito {str(n_concursos_ambito[1][0]).lower()}, que representan un {n_concursos_ambito[1][1]} del total. La duración media de los concursos 
      es de {int(ejecutor_querys(cur, query_duracion_concursos)[0][0])} días. 
      
      El mes que más concursos se han realizado ha sido {meses[int(n_concursos_mes[0][0])]}, con un total de {n_concursos_mes[0][1]} concursos, y el mes que menos concursos se han realizado ha sido el mes de {meses[int(n_concursos_mes[-1][0])]}, 
      con un total de {n_concursos_mes[-1][1]} concursos. La provincia en la que más concursos se han realizado ha sido {concursos_provincias[0][0]}, y la provincia en la que menos se han realizado ha sido {concursos_provincias[-1][0]}.
      
      De media en un concurso se realizan {str(media_pruebas_concurso[0][0]).replace('.', ',')} pruebas. Nos encontramos con {len(tipos_pruebas)} tipos de pruebas diferentes, siendo las más comunes las siguientes: 
      - {prueba_norma[tipos_pruebas[0][0]]}
      - {prueba_norma[tipos_pruebas[1][0]]}
      - {prueba_norma[tipos_pruebas[2][0]]}
      - {prueba_norma[tipos_pruebas[3][0]]}
      
      de media 

      """)




      Se han celebrado un total de 122 concursos, entre el 2025-01-10 y el 2025-04-27. Un 73 % de los concursos han sido de 
      ámbito internacional, frente a los de ámbito nacional, que representan un 26 % del total. La duración media de los concursos 
      es de 3 días. 

      El mes que más concursos se han realizado ha sido febrero, con un total de 41 concursos, y el mes que menos concursos se han realizado ha sido el mes de abril, 
      con un total de 21 concursos. La provincia en la que más concursos se han realizado ha sido Valencia, y la provincia en la que menos se han realizado ha sido Córdoba.

      De media en un concurso se realizan 13 pruebas. Nos encontramos con 14 tipos de pruebas diferentes, siendo las más comunes las siguientes: 
      - Baremo A con cronómetro
      - Baremo A con cronómetro y desempate
      - Dos fases, ambas con cronómetro
      - Dos fases especial

      de media 

      


In [21]:
concursos = pd.DataFrame(ejecutor_querys(cur, query_concursos_completos)).rename(columns = {0: "id_concurso", 1: "nombre_concurso", 2: "categoria_concurso", 3: "pais_concurso", 4: "provincia_concurso", 5: "localidad_concurso", 6: "ambito_concurso", 7: "federacion_concurso", 8: "fecha_inicio_concurso", 9: "fecha_fin_concurso"})
concursos.head()

,id_concurso,nombre_concurso,categoria_concurso,pais_concurso,provincia_concurso,localidad_concurso,ambito_concurso,federacion_concurso,fecha_inicio_concurso,fecha_fin_concurso
0,1,CSN3* Invierno,CSN***,Epaña,Sevilla,Real Club Pineda,Nacional,Federación Hípica Andaluza,2025-01-10,2025-01-12
1,2,SPRING MET I 2025 YH*,CSI YH*,Epaña,Valencia,Centro Ecuestre Oliva Nova,Internacional,Federación Hípica Comunidad Valenciana,2025-01-14,2025-01-16
2,3,MOURA TOURS VALENCIA - SPRING TOUR,CSI*,Epaña,Valencia,Moura Tours,Internacional,Federación Hípica Comunidad Valenciana,2025-01-20,2025-01-26
3,4,SPRING MET I 2025 - SILVER 1*,CSI*,Epaña,Valencia,Centro Ecuestre Oliva Nova,Internacional,Federación Hípica Comunidad Valenciana,2025-01-16,2025-01-18
4,5,SPRING MET I 2025 - GOLD 2*,CSI**,Epaña,Valencia,Centro Ecuestre Oliva Nova,Internacional,Federación Hípica Comunidad Valenciana,2025-01-16,2025-01-19


In [22]:
df = pd.DataFrame(n_concursos_ambito)
# Convert percentage strings to integers
df[1] = df[1].str.replace('%', '').str.strip().astype(int)
colores = ['#4c78a8', '#54a24b']
fig = px.pie(df, values=1, names=0, title='Porcentaje de concursos por ámbito', color_discrete_sequence=colores)
fig.update_traces(textinfo='percent', textfont_color='white')
fig.update_layout(width=600, height=400, title_x=0.5, title_font=dict(size = 16, weight='bold'))
fig.write_image("concursos_por_ambito.png")
fig.show()

In [23]:
# Suponiendo que tu DataFrame se llama df
diccionario = concursos.groupby('provincia_concurso')['localidad_concurso'].unique().apply(list).to_dict()
diccionario

{'Asturias': ['Club Hipico Astur'],
 'Barcelona': ['Real Club De Polo De Barcelona', 'Open Sports Club'],
 'Cantabria': ['Heras Horses & Events'],
 'Cádiz': ['Dehesa Montenmedio'],
 'Córdoba': ['Palma Del Rio'],
 'Granada': ['Granada'],
 'Madrid': ['Club Hipico Las Cadenas',
  'R.A.C.E.',
  'Las Cadenas',
  'R.S.H.E.C.C.'],
 'Murcia': ['Club Aros'],
 'Segovia': ['Cenro Ecuestre De Castilla Y León'],
 'Sevilla': ['Real Club Pineda'],
 'Tenerife': ['C.H Nueva Atalaya', 'C.H. Del Sur Los Migueles'],
 'Valencia': ['Centro Ecuestre Oliva Nova',
  'Moura Tours',
  'Ceon',
  'Mouratours Valencia',
  'Mouratours'],
 'Zamora': ['Equus Duri']}

In [24]:
query1 = """
        SELECT *
        FROM concursos c
        WHERE c.nombre_concurso = 'CSN2* LAS CADENAS';
"""
cur.execute(query1)
cosita = pd.DataFrame(cur.fetchall())
cosita

,0,1,2,3,4,5,6,7,8,9
0,20,CSN2* LAS CADENAS,CSN**,Epaña,Madrid,Club Hipico Las Cadenas,Nacional,Federación Hípica de Madrid,2025-01-25,2025-01-26
1,46,CSN2* LAS CADENAS,CSN**,Epaña,Madrid,Las Cadenas,Nacional,Federación Hípica de Madrid,2025-02-15,2025-02-16
2,100,CSN2* LAS CADENAS,CSN**,Epaña,Madrid,Club Hipico Las Cadenas,Nacional,Federación Hípica de Madrid,2025-03-29,2025-03-30


## Análisis de jinetes y caballos (binomios)

In [25]:
query_jinetes = """
        
        SELECT 
            DISTINCT COUNT(id_jinete)
        FROM jinetes j; 

"""
query_caballos = """
        
        SELECT 
            DISTINCT COUNT(id_caballo)
        FROM caballos c; 

"""

query_binomios = """

        SELECT 
            DISTINCT COUNT(CONCAT(j.id_jinete, '-', c.id_caballo))
        FROM resultados r
            JOIN jinetes j ON r.id_jinete = j.id_jinete
            JOIN caballos c ON r.id_caballo = c.id_caballo
"""

query_resultados_jinete = """

        SELECT 
            j.nombre_jinete,
            ROUND(AVG(r.puesto))
        FROM resultados r
            JOIN jinetes j ON r.id_jinete = j.id_jinete
            JOIN resultados_salto rs ON r.id_resultado = rs.id_resultado
        GROUP BY j.nombre_jinete
        ORDER BY 2 ASC;

"""
tipos_categorias = """
        SELECT 
            DISTINCT categoria_concurso
        FROM concursos c;
"""

query_resultados_jinete_2 = """
        SELECT 
            c.categoria_concurso,
            p.nombre_prueba

        FROM resultados r
        JOIN jinetes j ON r.id_jinete = j.id_jinete
        JOIN concursos c ON r.id_concurso = c.id_concurso
        JOIN resultados_salto rs ON r.id_resultado = rs.id_resultado
        JOIN pruebas p ON r.id_prueba = p.id_prueba
        GROUP BY c.categoria_concurso, p.nombre_prueba

        ;
"""
dinero_concurso_categoria = """
        WITH DineroTotalPorConcurso AS (
        SELECT
            r.id_concurso,
            SUM(r.dinero_premio) AS total_premio_concurso
        FROM resultados r
        GROUP BY r.id_concurso
        )
        SELECT
            c.categoria_concurso,
            AVG(dtpc.total_premio_concurso) AS media_dinero_premio_por_concurso
        FROM concursos c
            JOIN DineroTotalPorConcurso dtpc ON c.id_concurso = dtpc.id_concurso
        GROUP BY c.categoria_concurso
        ORDER BY c.categoria_concurso ;
        """
caballos_medios_jinete = """ 
    SELECT
    AVG(caballos_distintos) AS promedio_caballos_por_jinete
    FROM (
        SELECT
            j.id_jinete,
            COUNT(DISTINCT r.id_caballo) AS caballos_distintos
        FROM
            resultados r
        JOIN
            jinetes j ON r.id_jinete = j.id_jinete
        GROUP BY
            j.id_jinete
    );
"""


In [26]:
conn.rollback()

en cada tipo de prueba, que jinete es mejor/ al revés, y ver que jinete como overall es mejor y el mejor caballo, hacer un top 10. Ver si hay correlacion entre ganar y haber sido/mas rapido/tener mas o menos puntos en cada tipo de prueba. Dividir por categoría

In [27]:
coso = ejecutor_querys(cur, caballos_medios_jinete)
round(coso[0][0])

3

In [28]:
categorias_premios = pd.DataFrame(ejecutor_querys(cur , dinero_concurso_categoria))
categorias_premios[1] = categorias_premios[1].apply(lambda x: round(x, 2))
categorias_premios = categorias_premios.rename(columns = {0: "categoria", 1: "premio_medio_repartido"})
categorias_premios.sort_values(by = "premio_medio_repartido", ascending = False, inplace = True)
categorias_premios

,categoria,premio_medio_repartido
4,CSI****,375993.00
3,CSI***,191423.87
2,CSI**,95267.17
10,CSN*****,37170.00
1,CSI*,30654.96
0,CSI YH*,27653.00
9,CSN****,26215.00
5,CSIO Ch/J/Y,22900.00
8,CSN***,11167.21
7,CSN**,5542.94


In [29]:
query_binomios = """
    SELECT 
        COUNT(DISTINCT (id_jinete, id_caballo))
    FROM resultados r
"""


In [30]:
conn.rollback()

In [31]:
ejecutor_querys(cur, query_binomios)

[(9012,)]

In [32]:
print(f"""
      Ha habido una participación de un total de {ejecutor_querys(cur, query_binomios)[0][0]} binomios en los concursos. A continuación realizamos un análisis de los binomios, en función
      de los resultados obtenidos en los concursos.
      """)


      Ha habido una participación de un total de 9012 binomios en los concursos. A continuación realizamos un análisis de los binomios, en función
      de los resultados obtenidos en los concursos.
      


In [33]:
pd.set_option('display.max_columns', None)

- CSNCJ:nacional caballos jovenes
- CSIO Ch/J/Y: internacional children, junior, young rider
- CSI YH: internacional young horse

## Dividimos entre categorías nacionales e internacionales

In [34]:
conn.rollback()

In [35]:
df = pd.DataFrame(ejecutor_querys(cur, query_resultados_jinete_2))
diccionario_agrupado = df.groupby(0)[1].apply(list).to_dict()

- uno general, que selecciono el concurso y puedo ver métricas del mismo, puedo ver los resultados por ej, y métricas. En esta página tendré una visión general de los concursos, y luego por concurso, los filtros serán concurso y fecha, de manera que puedo filtrar por concurso y por fecha. 
- uno de binomios, en el que elijo el jinete con x caballo, y otro jinete con x caballo para comparar rendimientos. calculo por binomio la media de puntos de obstáculos que hacen en una salida a pista, la media de puntos de tiempo que hacen en una salida a pista, y el tiempo. mirar si filtrar por ronda. 


In [36]:
diccionario_agrupado

{'CSI YH*': ['5 años - 1,20 - A.c.c. y desempate (238.2.2)',
  '6 años - 1,30 - A c.c. (238.2.1)',
  '1,10 - A c.c. (238.2.1)',
  '1,35 (G) - Dos Fases (274.1.5.3)',
  '6 años - 1,20 - 2 fases Asc / Acc (274.1.5.2)',
  '5 años - 1,15 - A.c.c. y desempate (238.2.2)',
  '7 años - 1,30 - A c.c. (238.2.1)',
  '8 años - 1,45 - A.c.c y desempate (238.2.2)',
  '8 años - 1,45 - A.c.c. (238.2.1)',
  '6 años - 1,25 - Dos Fases Especial (274.2)',
  '1,40 (G) - Dos Fases (274.1.5.3)',
  '7 años - 1,25 - A c.c. (238.2.1)',
  '1,20 - A c.c. (238.2.1)',
  '7 años - 1,30 - Dos Fases (274.1.5.3)',
  '7 años - 1,35 - A sin cronómetro (238.1.1)',
  '5 años - 1,10 - Dos Fases (274.1.5.3)',
  '5 años - 1,15 - Dos Fases (274.1.5.3)',
  '6 años - 1,25 - A.c.c. y desempate (238.2.2)',
  '7 años - 1,35 - Dos Fases (274.1.5.3)',
  '1,20 - A.c.c. y desempate (238.2.2)',
  '7 años - 8 años - A.c.c (238.2.1)',
  '1,35 (G) - A.c.c. y desempate (238.2.2)',
  '8 años - 1,40 - A.c.c. (238.2.1)',
  '5 años - 1,05 - A s



caballos y jinetes, categoría en los que han participado, y altura de obstáculos, cantidad de veces que pasa a jump off, top caballos por edad, 5, 6, 7, años (nº rounds hechos, cantidad de doble 0s y ya sacar %). top caballos x competicion: 1,40, 1,50, 1,60, 5*, 6*.

comparar performance de un jinete con diferentes caballos

### Análisis caballos 5 años

Dentro de las pruebas de caballos de 5 años nos encontramos solo con las siguientes:
- A.c.c. y desempate (238.2.2)',
- Dos Fases Especial (274.2)',
- A sin cronómetro (238.1.1)',
- A c.c. (238.2.1)',
- Dos Fases (274.1.5.3)',
- A.c.c. y desempate (238.2.2)',

Son pruebas que tendrán solo dos rondas como mucho, ya sea con desempate o sin desempate. Las alturas de los obstáculos se encuentran entre 1,05 y 1,20. 

In [37]:
query = """ 
    SELECT 
        j.nombre_jinete,
        c.nombre_caballo,
        COUNT(r.id_resultado)
    FROM resultados r
        JOIN pruebas p ON r.id_prueba = p.id_prueba
        JOIN jinetes j ON r.id_jinete = j.id_jinete
        JOIN caballos c ON r.id_caballo = c.id_caballo
        JOIN resultados_salto rs ON r.id_resultado = rs.id_resultado
    GROUP BY j.nombre_jinete, c.nombre_caballo;
"""
query_2 = """ 

    SELECT 
        nombre_prueba
    FROM pruebas
    WHERE nombre_prueba LIKE '%5 años%';
"""

query_4 = """ 
    SELECT 
        COUNT(DISTINCT c.id_caballo)
    FROM resultados r
        JOIN pruebas p ON r.id_prueba = p.id_prueba
        JOIN jinetes j ON r.id_jinete = j.id_jinete
        JOIN caballos c ON r.id_caballo = c.id_caballo
        JOIN resultados_salto rs ON r.id_resultado = rs.id_resultado
    WHERE p.nombre_prueba LIKE '%5 años%';
"""

query_5 = """
    SELECT
        CASE
            WHEN p.nombre_prueba LIKE '%5 años%' THEN '5 años'
            WHEN p.nombre_prueba LIKE '%6 años%' THEN '6 años'
            WHEN p.nombre_prueba LIKE '%7 años%' THEN '7 años'
            ELSE 'Otra edad'
        END AS edad_referenciada,
        COUNT(DISTINCT c.id_caballo) AS numero_caballos
    FROM resultados r
        JOIN pruebas p ON r.id_prueba = p.id_prueba
        JOIN caballos c ON r.id_caballo = c.id_caballo
    GROUP BY
        edad_referenciada;
"""

In [38]:
conn.rollback()

In [39]:
import numpy as np

In [40]:
query_cinco = """ 
    SELECT 
        COUNT(DISTINCT c.id_caballo)
    FROM resultados r
        JOIN pruebas p ON r.id_prueba = p.id_prueba
        JOIN jinetes j ON r.id_jinete = j.id_jinete
        JOIN caballos c ON r.id_caballo = c.id_caballo
        JOIN resultados_salto rs ON r.id_resultado = rs.id_resultado
    WHERE p.nombre_prueba LIKE '%5 años%';
"""
query_6 = """ 
    SELECT 
        COUNT(DISTINCT c.id_caballo)
    FROM resultados r
        JOIN pruebas p ON r.id_prueba = p.id_prueba
        JOIN jinetes j ON r.id_jinete = j.id_jinete
        JOIN caballos c ON r.id_caballo = c.id_caballo
        JOIN resultados_salto rs ON r.id_resultado = rs.id_resultado
    WHERE p.nombre_prueba LIKE '%6 años%';
"""
query_7 = """ 
    SELECT 
        COUNT(DISTINCT c.id_caballo)
    FROM resultados r
        JOIN pruebas p ON r.id_prueba = p.id_prueba
        JOIN jinetes j ON r.id_jinete = j.id_jinete
        JOIN caballos c ON r.id_caballo = c.id_caballo
        JOIN resultados_salto rs ON r.id_resultado = rs.id_resultado
    WHERE p.nombre_prueba LIKE '%7 años%';
"""
query_todos = """ 
    SELECT 
        COUNT(DISTINCT c.id_caballo)
    FROM resultados r
        JOIN pruebas p ON r.id_prueba = p.id_prueba
        JOIN jinetes j ON r.id_jinete = j.id_jinete
        JOIN caballos c ON r.id_caballo = c.id_caballo
        JOIN resultados_salto rs ON r.id_resultado = rs.id_resultado;
"""

In [41]:
conn.rollback()

In [42]:
cinco_años = ejecutor_querys(cur, query_cinco)
seis_años = ejecutor_querys(cur, query_6)
siete_años = ejecutor_querys(cur, query_7)
todos = ejecutor_querys(cur, query_todos)
caballos_jovenes = sum([cinco_años[0][0], seis_años[0][0], siete_años[0][0]])
porcentaje_caballos_jovenes = round((caballos_jovenes / todos[0][0]) * 100, 2)
print(cinco_años, seis_años, siete_años, todos, caballos_jovenes, porcentaje_caballos_jovenes)

[(347,)] [(701,)] [(738,)] [(8332,)] 1786 21.44


In [43]:
query_top_10_5_años = """ 
    SELECT 
        j.nombre_jinete,
        c.nombre_caballo,
        COUNT(r.id_resultado),
        SUM(CASE WHEN r.puesto BETWEEN 1 AND 3 THEN 1 ELSE 0 END)
    FROM resultados r
        JOIN pruebas p ON r.id_prueba = p.id_prueba
        JOIN jinetes j ON r.id_jinete = j.id_jinete
        JOIN caballos c ON r.id_caballo = c.id_caballo
        JOIN resultados_salto rs ON r.id_resultado = rs.id_resultado
    WHERE p.nombre_prueba LIKE '%5 años%'
    GROUP BY j.nombre_jinete, c.nombre_caballo
    ORDER BY 4 DESC;
"""


query_top_10_6_años = """ 
    SELECT 
        j.nombre_jinete,
        c.nombre_caballo,
        COUNT(r.id_resultado),
        SUM(CASE WHEN r.puesto BETWEEN 1 AND 3 THEN 1 ELSE 0 END)
    FROM resultados r
        JOIN pruebas p ON r.id_prueba = p.id_prueba
        JOIN jinetes j ON r.id_jinete = j.id_jinete
        JOIN caballos c ON r.id_caballo = c.id_caballo
        JOIN resultados_salto rs ON r.id_resultado = rs.id_resultado
    WHERE p.nombre_prueba LIKE '%6 años%'
    GROUP BY j.nombre_jinete, c.nombre_caballo
    ORDER BY 4 DESC;
"""


query_top_10_7_años = """ 
    SELECT 
        j.nombre_jinete,
        c.nombre_caballo,
        COUNT(r.id_resultado),
        SUM(CASE WHEN r.puesto BETWEEN 1 AND 3 THEN 1 ELSE 0 END)
    FROM resultados r
        JOIN pruebas p ON r.id_prueba = p.id_prueba
        JOIN jinetes j ON r.id_jinete = j.id_jinete
        JOIN caballos c ON r.id_caballo = c.id_caballo
        JOIN resultados_salto rs ON r.id_resultado = rs.id_resultado
    WHERE p.nombre_prueba LIKE '%7 años%'
    GROUP BY j.nombre_jinete, c.nombre_caballo
    ORDER BY 4 DESC;
"""

query_top_10 = """ 
    SELECT 
        j.nombre_jinete,
        c.nombre_caballo,
        COUNT(r.id_resultado),
        SUM(CASE WHEN r.puesto BETWEEN 1 AND 3 THEN 1 ELSE 0 END)
    FROM resultados r
        JOIN pruebas p ON r.id_prueba = p.id_prueba
        JOIN jinetes j ON r.id_jinete = j.id_jinete
        JOIN caballos c ON r.id_caballo = c.id_caballo
        JOIN resultados_salto rs ON r.id_resultado = rs.id_resultado
    WHERE p.nombre_prueba NOT LIKE '%7 años%' AND p.nombre_prueba NOT LIKE '%5 años%' AND p.nombre_prueba NOT LIKE '%6 años%'
    GROUP BY j.nombre_jinete, c.nombre_caballo
    ORDER BY 4 DESC;
"""


In [44]:
resultados = pd.DataFrame(ejecutor_querys(cur, query_top_10))
# resultados_5_años["coso"] = round((resultados_5_años[3]/resultados_5_años[2])*100, 2)
resultados


,0,1,2,3
0,oliver fletcher,krack bleu c,18,15
1,james smith,dominican 2000 z,47,15
2,charlotte bevan,cupido quality z,24,15
3,sabine comet,flipper du chateau,18,15
4,james smith,juno rose 23,33,14
...,...,...,...,...
7422,jackson reed stephenson,domination f z,17,0
7423,leigh ofer -,carambola,5,0
7424,georgie tiplady,premier george,8,0
7425,ugo berrittella,cornetto quality z,9,0


In [45]:
resultados_5_años = pd.DataFrame(ejecutor_querys(cur, query_3))
resultados_5_años["doble_cero"] = resultados_5_años.apply(lambda x: 1 if  ((pd.isna(x[2]) or x[2] == 0) and (pd.isna(x[3]) or x[3] == 0)) else 0,axis=1)
resultados_5_años[pd.isna(resultados_5_años[2])]

NameError: name 'query_3' is not defined

In [ ]:
resultados_5_años_unicos = resultados_5_años.groupby([0, 1]).agg({"doble_cero": "sum", 4: "count"}).reset_index()
resultados_5_años_unicos.sort_values(by = 4, ascending= False)

,0,1,doble_cero,4
318,rudie wallerbosch,cosi celeste du perchet,25,25
49,bernardo moura,extasy de vilamoura z,13,22
29,ana gonzález menéndez,diamant de chalina,8,21
7,alejandro fernández sanchez,quiron del amor,16,20
40,aurelia pollet,kamelia gesmeray,18,18
...,...,...,...,...
245,marco pellegrino,kemerald de kreisker,1,1
237,luis mateos bernaldez,perry,1,1
229,lorenzo pradas freire,hermes van’t keijtershof z,1,1
78,christopher smith,billy becks,1,1


In [ ]:
len(resultados_5_años[1].unique())

347

In [ ]:
resultados_5_años = pd.DataFrame(ejecutor_querys(cur, query_3))
# resultados_5_años = resultados_5_años.rename(columns = {0: 'jinete', 1: 'caballo', 2: 'n_concursos_realizados', 3: 'n_rondas_cero_obs', 4: 'n_rondas_cero_tmp', 5:'n_rondas_doble_cero'})
# resultados_5_años["rondas_doble_cero"] = resultados_5_años[4] - resultados_5_años[3]
# resultados_5_años = resultados_5_años.sort_values(by = "rondas_doble_cero", ascending = False)
resultados_5_años["doble_cero"] = resultados_5_años.apply(lambda x: 1 if 
                                                          ((pd.isna(x[3]) or x[3] == 0) and (pd.isna(x[4]) or x[4] == 0) and (pd.isna(x[5]) or x[5] == 0) and (pd.isna(x[6]) or x[6] == 0)) 
                                                          
                                                          else 0,axis=1)
resultados_5_años[resultados_5_años[0] == 'rudie wallerbosch']

GroupingError: la columna «p.nombre_prueba» debe aparecer en la cláusula GROUP BY o ser usada en una función de agregación
LINE 5:         p.nombre_prueba,
                ^
